In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py
import matplotlib as mpl
from tqdm import tqdm


from holodeck import plot, detstats
import holodeck.single_sources as sings
from holodeck.constants import YR, MSOL, MPC
import holodeck as holo

import hasasia.sim as hsim
import os


Set up

In [ ]:
SHAPE = None
NREALS = 500
# NREALS = 20
NFREQS = 40
NLOUDEST = 10

BUILD_ARRAYS = False
SAVEFIG = True
TOL=0.01
MAXBADS=5

NVARS = 21
# NVARS = 6

NPSRS = 40
NSKIES = 100
# NSKIES = 15

SHOW_GW=False

Get param names

In [ ]:
pspace = holo.param_spaces.PS_Uniform_09B(holo.log, nsamples=1, sam_shape=SHAPE, seed=None)
param_names = pspace.param_names

Get data and dets file functions

In [ ]:
def get_data(
        target, nvars=NVARS, nreals=NREALS, nskies=NSKIES, shape=SHAPE, red_gamma = None, red2white=None, gsc_flag=False,
    gw_only=False, cv=10,     
):
    if gw_only:
        path = '/Users/emigardiner/GWs/holodeck/output/anatomy_7GW'
    else:
        path = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz'

    load_data_from_file = path+f'/{target}_v{nvars}_r{nreals}_shape{str(shape)}/data_params.npz' 
    load_dets_from_file = path+f'/{target}_v{nvars}_r{nreals}_shape{str(shape)}/detstats_s{nskies}_cv{cv}' 

    if gsc_flag: load_dets_from_file = load_dets_from_file + '_gsc'

    if red_gamma is not None and red2white is not None:
        load_dets_from_file = load_dets_from_file+f'_r2w{red2white:.1f}_rg{red_gamma:.1f}'
    else:
        load_dets_from_file = load_dets_from_file+f'_white'
    load_dets_from_file = load_dets_from_file+'.npz'

    if os.path.exists(load_data_from_file) is False:
        err = f"load data file '{load_data_from_file}' does not exist, you need to construct it."
        raise Exception(err)
    if os.path.exists(load_dets_from_file) is False:
        err = f"load dets file '{load_dets_from_file}' does not exist, you need to construct it."
        raise Exception(err)
    file = np.load(load_data_from_file, allow_pickle=True)
    data = file['data']
    params = file['params']
    file.close()
    print(target, "got data")
    file = np.load(load_dets_from_file, allow_pickle=True)
    print(target, "loaded dets")
    print(file.files)
    dsdat = file['dsdat']
    file.close()

    return data, params, dsdat

def get_dp_arrays(
    target, nvars=NVARS, nreals=NREALS, nskies=NSKIES, shape=SHAPE, debug=False, 
    red=True, cv='midclbrt', gw_only=False,
    ):

    path = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz/figdata/dpboth'   
    filename = path+f'/dp_arrays_{target}_cv{cv}_v{nvars}_r{nreals}_s{nskies}_shape{str(shape)}'
    if gw_only:
        filename = filename+'_gw'
    filename = filename + '.npz'

    file = np.load(filename)
    if debug: print(f"{filename}\n{file.files}")
    xx = file['xx_params']
    yy_ss = file['yy_ss']
    ev_ss = file['ev_ss']
    yy_bg = file['yy_bg']
    # if red:
    #     y1p5 = file['y1p5_ratio']
    #     y3p0 = file['y3p0_ratio']
    file.close()
    # if red:
    #     return xx, y0p0, y1p5, y3p0, #y1p5, y3p0
    # else:
    return xx, yy_ss, ev_ss, yy_bg

In [ ]:
def build_arrays(
    target,
    gw_only = False,
    cv='midclbrt'):


    # white noise only
    data, params, dsdat = get_data(target, gw_only=gw_only, cv=cv) #gsc_flag=gsc_flag, )
    xx=[]
    yy_ss=[]
    ev_ss=[]
    yy_bg=[]
    for pp, par in enumerate(params):
        xx.append(params[pp][target])
        yy_ss.append(dsdat[pp]['dp_ss']) #  R,S
        ev_ss.append(dsdat[pp]['ev_ss']) #  R,S
        yy_bg.append(dsdat[pp]['dp_bg']) #  R

    # if red_flag:
    # # red_gamma = -1.5
    #     data, params, dsdat = get_data(target, red_gamma=-1.5, red2white=1.0, path=path)
    #     y1p5=[]
    #     for pp, par in enumerate(params):
    #         dp_bg = np.repeat(dsdat[pp]['dp_bg'], NSKIES).reshape(NREALS, NSKIES)
    #         dp_ss = dsdat[pp]['ev_ss']
    #         y1p5.append(dp_ss/dp_bg)

    #     # red_gamma = -3.0
    #     data, params, dsdat = get_data(target, red_gamma=-3.0, red2white=1, path=path)
    #     y3p0=[]
    #     for pp, par in enumerate(params):
    #         dp_bg = np.repeat(dsdat[pp]['dp_bg'], NSKIES).reshape(NREALS, NSKIES)
    #         dp_ss = dsdat[pp]['ev_ss']
    #         y3p0.append(dp_ss/dp_bg)

    path = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz'
    filename = path+f'/figdata/dpboth/dp_arrays_{target}_cv{cv}_v{NVARS}_r{NREALS}_s{NSKIES}_shape{str(SHAPE)}'
    if gw_only:
        filename = filename+'_gw'
    filename = filename +'.npz'

    np.savez(filename, xx_params = xx, yy_ss=yy_ss, ev_ss=ev_ss, yy_bg = yy_bg,)

In [ ]:
if BUILD_ARRAYS:
    for target in ['gsmf_phi0', 'gsmf_mchar0_log10',
                'mmb_mamp_log10', 'mmb_scatter_dex', ]:
        build_arrays(target, cv=10, gw_only=True)

In [ ]:
if BUILD_ARRAYS:
    for target in ['hard_time', 'gsmf_phi0', 'gsmf_mchar0_log10',
                'mmb_mamp_log10', 'mmb_scatter_dex', 'hard_gamma_inner']:
        build_arrays(target, cv=10)

# 1Col White Noise + GW Only

In [ ]:
# ylabel = plot.LABEL_EVSS
# targets = [
#     'gsmf_phi0', 'gsmf_mchar0_log10', 
#       'mmb_mamp_log10',  'mmb_scatter_dex',
#       'hard_time',  'hard_gamma_inner']
# col0p0 = [
#     '#336948', '#336948', 
#     '#9e5c41', '#9e5c41', 
#     '#2d839f', '#2d839f',
# ]
# # col1p5 = [
# #     '#4da169', '#e67739', '#35b6e0',
# #     '#4da169', '#e67739', '#35b6e0',
# # ]
# # col3p0 = [
# #     '#8fcf91', '#fda363', '#38cbff',
# #     '#8fcf91', '#fda363', '#38cbff',
# # ]
# col_gw = [
#     'k', 'k', 'k',
#     'k', 'k', 'k'
# ]

# fig, axs = plot.figax_single(nrows=3, ncols=2, sharey=True, sharex=False, xscale='linear',
#                              height=7)
# for ii, ax in enumerate(axs.flatten()):
#     ax.set_xlabel(plot.PARAM_KEYS[targets[ii]])
#     if ii == 2:
#         ax.set_ylabel(ylabel)

#     ax.yaxis.set_major_locator(mpl.ticker.LogLocator(numticks=999))
#     ax.yaxis.set_minor_locator(mpl.ticker.LogLocator(numticks=999, subs=(.2, .4, .6, .8)))

#     xx, yy0p0, = get_ratio_arrays(targets[ii], red=False)
#     yy0p0 *= 0.5

#     col=col0p0[ii]
#     for pp in [50, 95]:
#     # for pp in [50]: 
#         med, *conf = np.percentile(yy0p0, [50, 50-pp/2, 50+pp/2], axis=(1,2))
#         ax.plot(xx, med, alpha=0.9, color=col0p0[ii])
#         ax.fill_between(xx, *conf, color=col0p0[ii], alpha=0.25)

#     if ii<4:
#         xx_gw, yy_gw = get_ratio_arrays(targets[ii], red=False,
#                                         path='/Users/emigardiner/GWs/holodeck/output/anatomy_7GW/figdata')
#         yy_gw *= 0.5
#         for pp in [50]:
#             med, *conf = np.percentile(yy_gw, [50, 50-pp/2, 50+pp/2], axis=(1,2))
#             ax.plot(xx_gw, med, alpha=0.5, color=col_gw[ii], linestyle='--')
#             ax.fill_between(xx, *conf, color=col_gw[ii], alpha=0.2, linestyle='--')
#         print(f"{ii=}, {yy_gw.shape}")
#     else:
#         xx_gw = xx
#         yy_gw = yy_gw[int(NVARS/2)]
#         yy_gw = np.repeat(yy_gw, 21).reshape(NREALS*NSKIES, NVARS)
#         yy_gw = np.swapaxes(yy_gw, 0, 1).reshape(NVARS, NREALS, NSKIES,)
#         print(f"{ii=}, {yy_gw.shape}")

#         for pp in [50]:
#             med, *conf = np.percentile(yy_gw, [50, 50-pp/2, 50+pp/2], axis=(1,2))
#             ax.plot(xx_gw, med, alpha=0.5, color=col_gw[ii], linestyle='--')
#             ax.fill_between(xx, *conf, color=col_gw[ii], alpha=0.2, linestyle='--')
#         print(f"{ii=}, yy_gw.shape")

        

#         # med, *conf = np.percentile(yy1p5, [50, 50-pp/2, 50+pp/2], axis=(1,2))
#         # ax.plot(xx, med, alpha=0.9, color=col1p5[ii])
#         # ax.fill_between(xx, *conf, color=col1p5[ii], alpha=0.25)

#         # med, *conf = np.percentile(yy3p0, [50, 50-pp/2, 50+pp/2], axis=(1,2))
#         # ax.plot(xx, med, alpha=0.9, color=col3p0[ii])
#         # ax.fill_between(xx, *conf, color=col3p0[ii], alpha=0.25)

# # handles = [mpl.lines.Line2D([0], [0], label='$\gamma_\mathrm{red}=0.0$', color = col0p0[0]),
# #            mpl.lines.Line2D([0], [0], label='$\gamma_\mathrm{red}=-1.5$', color = col1p5[0]),
# #            mpl.lines.Line2D([0], [0], label='$\gamma_\mathrm{red}=-3.0$', color = col3p0[0])]
# # axs[0,0].legend(handles=handles, loc='upper left', ncol=2)
# plt.subplots_adjust(wspace=0.05, hspace=0.35)
# # fig.tight_layout()

# if SAVEFIG:
#     savename = '/Users/emigardiner/GWs/holodeck/output/figures/bigplots/evss'
#     # savename = savename+'/ratio_vs_var6_r2w_pp50.png'
#     savename = savename+f'/evss_vs_var{NVARS}_wn_gw_single.png'
#     fig.savefig(savename, dpi=100)

## 1 Col 

In [ ]:
xx, yy_ss, _, yy_bg = get_dp_arrays(targets[ii], red=False, cv=10)
print(f"{yy_ss.shape=}, {yy_bg.shape=}")

In [ ]:
ylabel = 'Detection Probability'
targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
      'mmb_mamp_log10',  'mmb_scatter_dex',
      'hard_time',  'hard_gamma_inner']
col_ss = [
    '#336948', '#336948', 
    '#9e5c41', '#9e5c41', 
    '#2d839f', '#2d839f',
]

col_bg = [
    '#193423', '#193423', 
    '#5b3425', '#5b3425', 
    '#164150', '#164150',
]

col_gwss = [
    'grey', 'grey',
    'grey', 'grey',
    'grey', 'grey', 
]

col_gwbg = [
    'black', 'black',
    'black', 'black',
    'black', 'black', 
]

fig, axs = plot.figax_single(nrows=3, ncols=2, sharey=True, sharex=False, xscale='linear',
                             height=7)
for ii, ax in enumerate(axs.flatten()):
    ax.set_xlabel(plot.PARAM_KEYS[targets[ii]])
    if ii == 2:
        ax.set_ylabel(ylabel)

    ax.yaxis.set_major_locator(mpl.ticker.LogLocator(numticks=999))
    ax.yaxis.set_minor_locator(mpl.ticker.LogLocator(numticks=999, subs=(.2, .4, .6, .8)))

    # get arrays
    xx, yy_ss, _, yy_bg = get_dp_arrays(targets[ii], red=False, cv=10)

    # plot single and background sources
    for pp in [50, 95]:
    # for pp in [50]: 
        med, *conf = np.percentile(yy_ss, [50, 50-pp/2, 50+pp/2], axis=(1,2))
        ax.plot(xx, med, alpha=0.9, color=col_ss[ii])
        ax.fill_between(xx, *conf, color=col_ss[ii], alpha=0.25)

        med, *conf = np.percentile(yy_bg, [50, 50-pp/2, 50+pp/2], axis=(1))
        ax.plot(xx, med, alpha=0.9, color=col_bg[ii], linestyle='--')
        ax.fill_between(xx, *conf, color=col_bg[ii], alpha=0.35)

    # add gw-only
    if ii<4:
        _, gw_ss, _, gw_bg = get_dp_arrays(targets[ii], red=False, gw_only=True, cv=10)       
        print(f"{gw_bg.shape=}")
        print(holo.utils.stats(np.std(gw_bg, axis=(1)))) 
        # print(f"{ii=}, {yy_gw.shape}")
    else:
        gw_ss = gw_ss[int(NVARS/2)]
        gw_ss = np.repeat(gw_ss, 21).reshape(NREALS*NSKIES, NVARS)
        gw_ss = np.swapaxes(gw_ss, 0, 1).reshape(NVARS, NREALS, NSKIES,)

        gw_bg = gw_bg[int(NVARS/2)]
        gw_bg = np.repeat(gw_bg, 21).reshape(NREALS, NVARS)
        gw_bg = np.swapaxes(gw_bg, 0, 1).reshape(NVARS, NREALS)
        print(f"{ii=}, {gw_ss.shape=}")
        print(f"{gw_bg.shape=}")

    for pp in [50]:
        med, *conf = np.percentile(gw_ss, [50, 50-pp/2, 50+pp/2], axis=(1,2))
        ax.plot(xx, med, alpha=0.95, color=col_gwss[ii], linestyle=':')
        ax.fill_between(xx, *conf, color=col_gwss[ii], alpha=0.3)

        med, *conf = np.percentile(gw_bg, [50, 50-pp/2, 50+pp/2], axis=(1))
        ax.plot(xx, med, alpha=0.6, color=col_gwbg[ii], linestyle=':', lw=2)
        ax.fill_between(xx, *conf, color=col_gwbg[ii], alpha=0.25)


plt.subplots_adjust(wspace=0.05, hspace=0.35)
# fig.tight_layout()

if SAVEFIG:
    savename = '/Users/emigardiner/GWs/holodeck/output/figures/bigplots/evss'
    savename = savename+f'/dpboth_cv10_bg.png'
    fig.savefig(savename, dpi=100)

In [ ]:
ylabel = plot.LABEL_EVSS +' or ' +plot.LABEL_DPBG
targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
      'mmb_mamp_log10',  'mmb_scatter_dex',
      'hard_time',  'hard_gamma_inner']


fig, axs = plot.figax_single(nrows=3, ncols=2, sharey=True, sharex=False, xscale='linear',
                             height=7)
for ii, ax in enumerate(axs.flatten()):
    ax.set_xlabel(plot.PARAM_KEYS[targets[ii]])
    if ii == 2:
        ax.set_ylabel(ylabel)

    ax.yaxis.set_major_locator(mpl.ticker.LogLocator(numticks=999))
    ax.yaxis.set_minor_locator(mpl.ticker.LogLocator(numticks=999, subs=(.2, .4, .6, .8)))

    xx, _, yy_ss, yy_bg = get_dp_arrays(targets[ii], red=False, cv=10)

    for pp in [50, 95]:
    # for pp in [50]: 
        med, *conf = np.percentile(yy_ss, [50, 50-pp/2, 50+pp/2], axis=(1,2))
        ax.plot(xx, med, alpha=0.9, color=col_ss[ii])
        ax.fill_between(xx, *conf, color=col_ss[ii], alpha=0.25)

        med, *conf = np.percentile(yy_bg, [50, 50-pp/2, 50+pp/2], axis=(1))
        ax.plot(xx, med, alpha=0.7, color=col_bg[ii], linestyle='--')
        ax.fill_between(xx, *conf, color=col_bg[ii], alpha=0.35)

    # add gw-only
    if ii<4:
        _, _, gw_ss, gw_bg = get_dp_arrays(targets[ii], red=False, gw_only=True, cv=10)        
        # print(f"{ii=}, {yy_gw.shape}")
    else:
        gw_ss = gw_ss[int(NVARS/2)]
        gw_ss = np.repeat(gw_ss, 21).reshape(NREALS*NSKIES, NVARS)
        gw_ss = np.swapaxes(gw_ss, 0, 1).reshape(NVARS, NREALS, NSKIES,)

        gw_bg = gw_bg[int(NVARS/2)]
        gw_bg = np.repeat(gw_bg, 21).reshape(NREALS, NVARS)
        gw_bg = np.swapaxes(gw_bg, 0, 1).reshape(NVARS, NREALS)
        print(f"{ii=}, {gw_ss.shape=}")

    for pp in [50]:
        med, *conf = np.percentile(gw_ss, [50, 50-pp/2, 50+pp/2], axis=(1,2))
        ax.plot(xx, med, alpha=0.5, color=col_gwss[ii], linestyle=':')
        ax.fill_between(xx, *conf, color=col_gwss[ii], alpha=0.3)

        med, *conf = np.percentile(gw_bg, [50, 50-pp/2, 50+pp/2], axis=(1))
        ax.plot(xx, med, alpha=0.9, color=col_gwbg[ii], linestyle=':')
        ax.fill_between(xx, *conf, color=col_gwbg[ii], alpha=0.6)

plt.subplots_adjust(wspace=0.05, hspace=0.35)
# fig.tight_layout()

if SAVEFIG:
    savename = '/Users/emigardiner/GWs/holodeck/output/figures/bigplots/evss'
    savename = savename+f'/dpev_cv10_bg.png'
    fig.savefig(savename, dpi=100)

# Lowest Var Calibration

In [ ]:
cvs = [20, 0, 0, 0, 0, 20]

for ii, target in enumerate(['hard_time', 'gsmf_phi0', 'gsmf_mchar0_log10', 
                'mmb_mamp_log10', 'mmb_scatter_dex', 'hard_gamma_inner']):
    build_arrays(target, cv=f"cv{cvs[ii]}")

In [ ]:
ylabel = plot.LABEL_EVSS +' (color) / ' +plot.LABEL_DPBG +' (black)'
targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
      'mmb_mamp_log10',  'mmb_scatter_dex',
      'hard_time',  'hard_gamma_inner']

cvs = [0, 0, 0, 0, 20, 20]

fig, axs = plot.figax_single(nrows=3, ncols=2, sharey=True, sharex=False, xscale='linear',
                             height=7)
for ii, ax in enumerate(axs.flatten()):
    ax.set_xlabel(plot.PARAM_KEYS[targets[ii]])
    if ii == 2:
        ax.set_ylabel(ylabel)

    ax.yaxis.set_major_locator(mpl.ticker.LogLocator(numticks=999))
    ax.yaxis.set_minor_locator(mpl.ticker.LogLocator(numticks=999, subs=(.2, .4, .6, .8)))

    xx, _, yy_ss, yy_bg = get_dp_arrays(targets[ii], cv=f"cv{cvs[ii]}", red=False, gsc_flag=True, nodiv_flag=True)

    print(f"{xx.shape=}, {yy_ss.shape=}, {yy_bg.shape=}")
    col=col0p0[ii]
    for pp in [50, 95]:
    # for pp in [50]: 
        med, *conf = np.percentile(yy_ss, [50, 50-pp/2, 50+pp/2], axis=(1,2))
        ax.plot(xx, med, alpha=0.9, color=col0p0[ii])
        ax.fill_between(xx, *conf, color=col0p0[ii], alpha=0.25)

        med, *conf = np.percentile(yy_bg, [50, 50-pp/2, 50+pp/2], axis=(1))
        ax.plot(xx, med, alpha=0.7, color='k')
        ax.fill_between(xx, *conf, color='k', alpha=0.25)


plt.subplots_adjust(wspace=0.05, hspace=0.35)
# fig.tight_layout()

if SAVEFIG:
    savename = '/Users/emigardiner/GWs/holodeck/output/figures/bigplots/evss'
    savename = savename+f'/dpev_lowestcv.png'
    fig.savefig(savename, dpi=100)